## Desafio 1 - Algoritmo de Moore

### Objetivo
- Minimizar o número de ordens atrasadas em problemas de máquina única (SMP – Single Machine Problem)

### Descrição
- Dado um conjunto de ‘n’ jobs a serem processados em uma única máquina, com tempos de processamento e datas de entrega conhecidos, definir a sequência que minimiza o número de jobs concluídos fora do prazo de entrega (atrasados)

### Entrada
- Tabela com tempos de processamento (processing times –  pi ) e datas de entrega prometidas (due dates –  di )

|i|p_i|d_i|
|---|---|---|
|0|3|7|
|1|4|8|
|2|2|9|
|3|6|12|
|4|1|4|

**Classe Job**

In [14]:
import pandas as pd

In [15]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.d=di     # due date
        self.C=0.0     # completion time
        self.T=0.0     # tardiness

**Função para cálculo dos indicadores**

In [16]:
def calcInd(jobs):     # Indicadores para SMP
    
    n=len(jobs)
    
    jobs[0].C=jobs[0].p
    for j in range(1,n):
        jobs[j].C=jobs[j-1].C+jobs[j].p
    
    for job in jobs:
        job.T=max(0,job.C-job.d)
        
    Cmax=jobs[-1].C
    
    Fbar=0.0
    TT=0.0
    Tmax=0.0
    nT=0
    for job in jobs:
        Fbar+=job.C
        TT+=job.T
        if job.T>0:
            nT+=1
            Tmax=max(Tmax,job.T)
    
    Fbar=Fbar/n
    
    # alternativa: usar o recurso "list comprehension" do Python
    # exemplo:
    # TT=sum([job.T for job in jobs])
    # nT=sum([1 if job.T>0 else 0 for job in jobs])

    return Cmax, Fbar,TT, Tmax, nT

**Função para printar os jobs e seus parâmetros**

In [17]:
def printJobs(jobs):

    tab=pd.DataFrame({
    "#":[job.i for job in jobs],
    "p":[job.p for job in jobs],
    "d":[job.d for job in jobs],
    "C":[job.C for job in jobs],
    "T":[job.T for job in jobs],
    })
    print(tab)
    

## Algoritmo de Moore

O algoritmo de Moore, cuja entrada são os tempos de processamento de cada job (p) e suas respectivas datas de entrega (d)

In [18]:
def AlgMoore(p,d):
    # Criação da instância
    jobs=[] 
    for i in range(len(p)):
        jobs.append(Job(i,p[i],d[i])) 
    # Criação das listas A (de Jobs) e B (de Backlog)
    A = jobs.copy()
    A.sort(key=lambda job: job.d)
    B = []
    # Calculo inicial dos indices
    calcInd(A)
    # implementação do algoritmo de moore
    while any([A[i].T != 0 for i in range(len(A))]) and any([A[i].d != 0 for i in range(len(A))]):
        FuncTard = list(filter(lambda job: job.T == 0, A))
        maxp = max(map(lambda job: job.p,FuncTard))
        for i in A:
            if i.p == maxp:
                elemento = i
        A.remove(elemento)
        B.append(elemento)
        calcInd(A)
    B.sort(key=lambda job:job.p)
    for i in range(len(B)):
        A.append(B[i])
    return A,calcInd(A)

**Testes**

In [19]:
# instância 1
p=[3,4,2,6,1]
d=[7,8,9,12,4]
A,a = AlgMoore(p,d)
printJobs(A)
print(f"Cmax = {a[0]} \nFbar=  {a[1]}\nTT=    {a[2]}\nTmax=  {a[3]}\nnT=    {a[4]}")

   #  p   d   C  T
0  4  1   4   1  0
1  0  3   7   4  0
2  2  2   9   6  0
3  3  6  12  12  0
4  1  4   8  16  8
Cmax = 16 
Fbar=  7.8
TT=    8.0
Tmax=  8
nT=    1


In [20]:
# instância 2
p=[3, 4, 2, 6, 1]
d=[20, 20, 20, 20, 20]
A,a = AlgMoore(p,d)
printJobs(A)
print(f"Cmax = {a[0]} \nFbar=  {a[1]}\nTT=    {a[2]}\nTmax=  {a[3]}\nnT=    {a[4]}")

   #  p   d   C  T
0  0  3  20   3  0
1  1  4  20   7  0
2  2  2  20   9  0
3  3  6  20  15  0
4  4  1  20  16  0
Cmax = 16 
Fbar=  10.0
TT=    0.0
Tmax=  0.0
nT=    0


In [21]:
# instância 3
p=[3, 4, 2, 6, 1]
d=[0, 0, 0, 0, 0]
A,a = AlgMoore(p,d)
printJobs(A)
print(f"Cmax = {a[0]} \nFbar=  {a[1]}\nTT=    {a[2]}\nTmax=  {a[3]}\nnT=    {a[4]}")

   #  p  d   C   T
0  0  3  0   3   3
1  1  4  0   7   7
2  2  2  0   9   9
3  3  6  0  15  15
4  4  1  0  16  16
Cmax = 16 
Fbar=  10.0
TT=    50.0
Tmax=  16
nT=    5


In [22]:
# instância 4
p=[1, 2, 3, 4, 5]
d=[10, 10, 10, 10, 10]
A,a = AlgMoore(p,d)
printJobs(A)
print(f"Cmax = {a[0]} \nFbar=  {a[1]}\nTT=    {a[2]}\nTmax=  {a[3]}\nnT=    {a[4]}")

   #  p   d   C  T
0  0  1  10   1  0
1  1  2  10   3  0
2  4  5  10   8  0
3  2  3  10  11  1
4  3  4  10  15  5
Cmax = 15 
Fbar=  7.6
TT=    6.0
Tmax=  5
nT=    2


In [23]:
# instância 5
p=[1, 6, 2, 1, 9, 8, 2, 3, 9, 1]
d=[20, 30, 30, 30, 30, 30, 30, 30, 30, 40]
A,a = AlgMoore(p,d)
printJobs(A)
print(f"Cmax = {a[0]} \nFbar=  {a[1]}\nTT=    {a[2]}\nTmax=  {a[3]}\nnT=    {a[4]}")

   #  p   d   C   T
0  0  1  20   1   0
1  1  6  30   7   0
2  2  2  30   9   0
3  3  1  30  10   0
4  5  8  30  18   0
5  6  2  30  20   0
6  7  3  30  23   0
7  9  1  40  24   0
8  8  9  30  33   3
9  4  9  30  42  12
Cmax = 42 
Fbar=  18.7
TT=    15.0
Tmax=  12
nT=    2
